# Goal:
Develop an object that will manage data and attributes for a vector flow set, while also facilitating trajectory calculation, etc.

For this purpose we will import all functions from `gbeflow`, so that this code will not need to be changed when it is added to the module.

In [2]:
from gbeflow import *

/Users/morganschwartz/anaconda3/lib/python3.7/site-packages/bebi103/viz.py:30: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  Features requiring DataShader will not work and you will get exceptions.""")


In [3]:
import tifffile
from czifile import CziFile

In [4]:
import bebi103

In [5]:
from scipy.interpolate import RectBivariateSpline

In [13]:
name = '20180110_htl_glc-CreateImageSubset-02_sc11_htl_rotate_brt'

In [8]:
class VectorField:
    
    def __init__(self,name):
        
        self.name = name
        
        # Import vector data as dataframe
        self.df = tidy_vector_data(self.name)
        
        # Transform vector dataframe into arrays
        self.tt,self.xx,self.yy,self.vx,self.vy = reshape_vector_data(self.df)
        
        # Extract unique x and y positions
        self.xval = np.unique(self.xx)
        self.yval = np.unique(self.yy)
        self.tval = np.unique(self.tt)
        
        # Initialize start point dataframe
        self.starts = pd.DataFrame()
        
    def add_image_data(self,impath):
        
        # Determine file type for import
        if impath[-3:] == 'czi':
            self.img = CziImport(impath,summary=False).data
        elif 'tif' in impath[-4:]:
            self.img = tifffile.imread(impath)
        else:
            print('Image files must be czi or tif')
            
    def pick_start_points(self,notebook_url='localhost:8888'):
        
        # Record start clicks
        p = bebi103.viz.record_clicks(self.img[0],
                                      notebook_url=notebook_url,
                                      flip=False)
        
        return(p)
    
    def save_start_points(self,p):
        
        # Add to starts dataframe
        self.starts = self.starts.append(p.to_df())
        
    def initialize_interpolation(self,dt,timer=True):
        
        self.dt = dt
        
        # Store interpolation object over time
        self.Ldx = []
        self.Ldy = []
        
        # Record interpolation initializationa as false
        self.interp_init = False
        
        # Set iterator with or without tqdm
        # Includes zeroth timepoint where all vx and vy = 0
        if timer == True:
            iterator = tqdm.tqdm(self.tval)
        else:
            iterator = self.tval

        for t in iterator:

            # Interpolate to find change in x and y
            dx = RectBivariateSpline(self.xval,self.yval,
                                     self.dt*self.vx[t])
            dy = RectBivariateSpline(self.xval,self.yval,
                                     self.dt*self.vy[t])
            
            # Save iterator to list
            self.Ldx.append(dx)
            self.Ldy.append(dy)
            
        # Set interpolation initialization value to True
        self.interp_init = True
        
    def calc_track(self,x0,y0,dt):
        
        # Check if interpolation has been initialized
        if hasattr(self,'interp_init') and (self.interp_init==True):
            # Continue with function without problem
            pass
        else:
            self.initialize_interpolation(dt)
            
        # Initialize position list with start value
        xpos = [x0]
        ypos = [y0]
        
        for t in range(np.max(self.tval)):
            
            # Calculate dx and dy from iterators
            dx = self.Ldx[t].ev(xpos[t],ypos[t])
            dy = self.Ldy[t].ev(xpos[t],ypos[t])
            
            # Update position arrays
            xpos.append(xpos[t]+dx)
            ypos.append(ypos[t]+dy)
            
        return(np.array([xpos,ypos]))
    
    def calc_track_set(self,starts,dt,name='',timer=True):
        
        # Check if track dataframe needs to be created
        if hasattr(self,'tracks') == False:
            self.tracks = pd.DataFrame()
        
        # Set up iterator
        if timer:
            iterator = tqdm.tqdm(starts.index)
        else:
            iterator = starts.index
            
        for i in iterator:
            x0,y0 = vf.starts.iloc[i]
            track = self.calc_track(x0,y0,dt)
            trackdf = pd.DataFrame({'x':track[0,:],'y':track[1,:],'t':self.tval,
                                            'track':[i]*track.shape[-1],'name':[name]*track.shape[-1]})
            self.tracks = pd.concat([self.tracks,trackdf])

Test initialization

In [11]:
%%time
vf = VectorField(name)

CPU times: user 20.9 s, sys: 2.34 s, total: 23.3 s
Wall time: 16.6 s


In [9]:
import os
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

Test image import

In [12]:
vf.add_image_data(os.path.join('../data',vf.name+'.tif'))

In [22]:
bokeh.io.show(bebi103.viz.imshow(vf.img[0]))

Test start point selection

In [27]:
p = vf.pick_start_points()

In [80]:
vf.save_start_points(p)

In [30]:
vf.starts

,x,y
0,987.404447,556.004286
1,1022.273438,500.107143
2,965.634816,493.682321
3,1021.347356,482.651250
4,1027.521234,426.203750
5,988.652644,430.044286
6,966.440104,395.527321


In [46]:
x0 = vf.starts.iloc[0].x
y0 = vf.starts.iloc[0].y
vf.calc_track(x0,y0,10)

100%|██████████| 166/166 [00:00<00:00, 183.44it/s]


array([[ 987.40444712,  987.40444712,  987.63344086,  988.37565835,
         989.04107423,  988.63072398,  991.89475054,  989.89627824,
         991.66903332,  993.48501973,  992.69709631,  994.03858315,
         998.33038686,  999.42946888, 1000.32064565, 1002.10096598,
        1003.64342796, 1003.7115071 , 1006.31227353, 1006.48696258,
        1002.01372657, 1005.85304397, 1004.17616461, 1003.8327302 ,
        1000.2893591 ,  994.84148723,  991.04400235,  988.63443199,
         983.82409943,  980.23688387,  974.12712782,  970.80515758,
         966.73144965,  965.67376447,  966.45416457,  964.32404081,
         963.55642847,  960.0533297 ,  959.99831311,  960.0170625 ,
         959.69306663,  959.86064619,  962.47555499,  965.75353356,
         968.83813843,  971.48621078,  973.22759917,  977.05794463,
         983.75339887,  986.17805758,  991.63171713,  994.45783207,
         997.00458592,  999.59209375, 1003.40114072, 1004.27921119,
        1008.15840278, 1014.1589544 , 1023.30506

Test batch track calculator

In [95]:
vf.calc_track_set(vf.starts,10,name='dt 10')


100%|██████████| 7/7 [00:00<00:00, 143.94it/s]

In [97]:
vf.calc_track_set(vf.starts,20,name='dt 20')


100%|██████████| 7/7 [00:00<00:00, 140.68it/s]

In [82]:
vf.tracks

,x,y,t,track,name
0,987.404447,556.004286,0,0,
1,987.404447,556.004286,1,0,
2,987.633441,552.556160,2,0,
3,988.375658,552.297783,3,0,
4,989.041074,549.791094,4,0,
5,988.630724,550.070065,5,0,
6,991.894751,549.497673,6,0,
7,989.896278,549.130525,7,0,
8,991.669033,551.849061,8,0,
9,993.485020,553.227141,9,0,


In [64]:
import matplotlib.pyplot as plt

In [1]:
fig,ax = plt.subplots(figsize=(12,10))
ax.imshow(vf.img[-1],cmap='Greys')
ax.scatter(vf.tracks.x,vf.tracks.y,c=vf.tracks.track,s=10)

NameError: name 'plt' is not defined